# 引入依赖

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np

from data_provider.datafeed.universe import Universe
from data_provider.nestlib.trading_cal import TradeCal
from data_provider.nestlib.mongo import getMongoClient
from smartbeta.backtest.factor_backtest import FactorBackTest

自定义因子回测
基于FactorBackTest, 我们可以对单个因子或多因子组合进行回测，其基本逻辑是根据因子排名来决定持股权重，详情参见因子回测文档。

更进一步，我们也可以通过自定义方法，额外添加回测逻辑，将研究者自身对市场的独到见解与因子选股技术强强联合！

添加自定义回测逻辑
格式

def my_market_opinion(trading_day):
    """
    Parameters
    ----------
    trading_day: str
        交易日

    Returns
    ---------
    ret: pd.Series类型
        index为股票ticker, value为处理后的因子值
    """
    raise NotImplementedError

In [ ]:
def load_one_day_factor_se(factor_name, trading_day):
    mongo_client = getMongoClient()

    factor_collection = mongo_client.factor_data.get_collection(factor_name.lower())
    factor_data = factor_collection.find({'td': int(trading_day)}, {'_id': False})
    ret = []
    for factor in factor_data:
        [x.update({'td': str(factor['td'])}) for x in factor['fator_info']]
        ret += factor['fator_info']

    one_day_factor_data = pd.DataFrame(data=ret, index=range(len(ret)))
    one_day_factor_data = one_day_factor_data.rename(
        columns={'td': 'tdate', 'tk': 'security_code', 'fv': 'factor_value'})

    # 过滤掉nan值和无穷大无穷小
    one_day_factor_data = one_day_factor_data[~one_day_factor_data.isin([np.nan, np.inf, -np.inf]).any(1)]
    factor_se = one_day_factor_data.set_index('security_code')['factor_value']
    return factor_se


def get_little_mkt(trading_day):
    mkt_df = Universe().get_market_value(trading_day, trading_day)
    mean_mk = mkt_df['total_market_value'].mean()
    return mkt_df[mkt_df['total_market_value'] < mean_mk].set_index('ticker')['total_market_value']


def get_big_mkt(trading_day):
    mkt_df = Universe().get_market_value(trading_day, trading_day)
    mean_mk = mkt_df['total_market_value'].mean()
    return mkt_df[mkt_df['total_market_value'] > mean_mk].set_index('ticker')['total_market_value']

def buy_big_mktcap_and_low_pe(trading_day):
    """
    自定义回测逻辑：
        - 筛选出大盘股
        - 进一步筛选出市盈率最低的10%
    """
    big_mkt = get_big_mkt(trading_day)
    pe_se = load_one_day_factor_se('ep_ttm', trading_day)
    
    pe_se = pe_se[pe_se.index.isin(big_mkt.index)]
    pe_se = pe_se[pe_se >= pe_se.quantile(0.9)]
    return pe_se